In [7]:
# pip install keybert
# pip install kex
# pip install spacy

In [12]:
from keybert import KeyBERT
import kex
import pandas as pd
from segtok.segmenter import split_multi
from segtok.tokenizer import web_tokenizer, split_contractions
from nltk.stem.porter import PorterStemmer
__stemmer = PorterStemmer()
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
from _stopwords import get_stopwords_list
from itertools import chain
import string
import spacy

import io
import os

import pandas as pd
  
#nlp =spacy.load('en_core_web_sm')
kw_model = KeyBERT()
from nltk.stem import SnowballStemmer

[nltk_data] Downloading package punkt to /Users/jennifer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2023-12-05 14:37:57 INFO     Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2023-12-05 14:37:57 INFO     Use pytorch device: cpu


In [13]:
BCpolitics = pd.read_csv("aws/BCpolitics1.csv")
CanadianPolitics = pd.read_csv("aws/CanadianPolitics.csv")
vancouver = pd.read_csv("vancouver.csv")

## housing affordability

In [45]:
house_list= ["Affordable housing","Affordable living","Reasonable housing","Inexpensive housing","Low-cost housing"
             ,"Economical housing","Attainable housing"]
house_list = list(map(lambda x: __stemmer.stem(x), house_list)) 
house_list = list(dict.fromkeys(house_list))
def sim_score(keyword_list, topic): 
    scores = []
    for keyword in keyword_list:
        
        if keyword is not None:
            for i in keyword:
                key = __stemmer.stem(i[0])
                if key in topic:
                    score = 1
                else:
                    score =0
        else:
            score= 0
        scores.append(score)
    return scores

In [63]:
def append_kex_list(text):
    ans = []
    keys_stemmed = [k['stemmed'] for k in text]
    ans.append(keys_stemmed)
    return ans

In [93]:
def append_keybert_list(text):
    ans = []
    keys = []
    for j in range(len(text)):
        keys.append(text[j][0])
    keys_stemmed = list(map(lambda x: __stemmer.stem(x), keys)) 
    keys_stemmed = list(dict.fromkeys(keys_stemmed))
    ans.append(keys_stemmed)
    return ans

In [83]:
def keyword(df, model_name):
    if(model_name == "keybert"):
        df["keyword"] = df["title"].apply(lambda x: kw_model.extract_keywords(x, keyphrase_ngram_range=(1,2), stop_words='english',
                          use_mmr=True, diversity=0.7)if not pd.isnull(x)else None)
        df["keyword"] =df["keyword"].apply(lambda x: append_keybert_list(x)if not x is None else None)
    elif(model_name == "TopicRank"):
        model = kex.TopicRank()
        df["keyword"] =df["title"].apply(lambda x: model.get_keywords(x)if not pd.isnull(x)else None)
        df["keyword"] =df["keyword"].apply(lambda x: append_kex_list(x)if not x is None else None)
    elif(model_name == "TFIDF"):
        model = kex.TFIDF()
        model.train(df["title"][0], export_directory='./tmp')
        df["keyword"] =df["title"].apply(lambda x: model.get_keywords(x, n_keywords=1000)if not pd.isnull(x)else None)
        df["keyword"] =df["keyword"].apply(lambda x: append_kex_list(x)if not x is None else None)
        